In [42]:
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [43]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import math
import time

In [ ]:
EDGES = {
    (0, 1): 'm', # coordinate in index 0 which is nose is connected to that in index 1 which is left eye
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

COLORS = {
    "m":(255,0,255),
    "c":(63,127,0),
    "y":(0,95,191)
}

# Draw Keypoints

In [44]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    # np.multiply(keypoints, [y,x,1]) will get the real coordinate as tflite returns normalized coordinate.
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

# Draw Connections

In [45]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
#     shaped_2 = np.squeeze(np.multiply(keypoints, [y,x,1]))
    shaped = np.multiply(np.squeeze(keypoints), [y,x,1])
#     print(shaped==shaped_2)
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold): 
            cTuple = COLORS[color]
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), cTuple, 2)

In [46]:
# draw_connections(frame, keypoints_with_scores, EDGES, 0.4)

# Find Angle

In [47]:
def calculate_angle(s,e,w):
    cSquare = (s[1]-w[1])**2 + (s[0]-w[0])**2
    aSquare = (s[1]-e[1])**2 + (s[0]-e[0])**2
    bSquare = (e[1]-w[1])**2 + (e[0]-w[0])**2
    angleRad = np.arccos((aSquare+bSquare-cSquare)/(2*np.sqrt(aSquare)*np.sqrt(bSquare)))
    angleDegree = math.degrees(angleRad)
    if angleDegree >180.0:
        angleDegree = 360-angleDegree
        
    return angleDegree 

# Make Detection

In [48]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
interpreter.allocate_tensors()

In [57]:
cap = cv2.VideoCapture(1)
pTime = 0
while cap.isOpened():
    ret, frame = cap.read()
    # Input: A frame of video or an image, 
    # represented as an float32 tensor of shape: 192x192x3. Channels order: RGB with values in [0, 255].
    # Reshape image to 192*192*3
    img = frame.copy()
#     img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256,256)
    input_image = tf.cast(img, dtype=tf.float32) # float32 as the document says.
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    kp = np.squeeze(keypoints_with_scores)
    if (kp[6,-1]>0.3 and kp[8,-1]>0.3 and kp[10,-1]>0.3):
        word = str(calculate_angle(kp[6],kp[8],kp[10]))
    else:
        word = "No right arm detected!!"
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.3)
    draw_keypoints(frame, keypoints_with_scores, 0.3)
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
#     print(fps)
    
    cv2.putText(frame, word, 
                   (10,10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    
    cv2.putText(frame, str(int(fps)), 
                   (250,10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

6.088228120689797e-10
4.902046812673339
5.714458548654529
5.988030517611586
4.948838570297558
6.28928305270529
6.284138121157912
6.413024498913657
5.915535547055704
5.6833002486433015
5.654392554106485
6.902056808663642
5.552486655934932
5.987979224902706
6.0224742333173475
5.901361266857549
6.009918369042996
6.466853226639531
6.277535564884868
6.211418763522629
6.137389120898095
6.03359811726255
6.785538176805948
6.1071394770139795
5.988723055198204
5.649571463784885
5.944923440208525
5.651048143602208
5.97351563056327
6.251701430602797
5.988081811199214
5.846265783703844
6.154364454591741
6.846937792716392
6.024195575395372
5.931622988817849
6.462528831975643
6.134848651060064
5.933947529257305
6.27567581316779
6.2482183291621975
6.097045603874847
6.053801898856587
6.158168598846565
6.1897491370544895
5.927306026929556
5.494551042303929
6.191201868149062
5.987936481649889
7.042350100657675
5.917162902049413
6.622494639547038
5.735376806362334
5.966887172264664
6.172914437597594
5.390

In [51]:
# plt.imshow(frame)

In [52]:
# plt.imshow(tf.cast(tf.squeeze(img), dtype=tf.int32))
# tf.image.resize_with_pad pads the picture with black element.

In [53]:
interpreter.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 256, 256,   3]),
  'shape_signature': array([  1, 256, 256,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [54]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 312,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [55]:
keypoints_with_scores[0][0] # each labels the particular landmark.

right_eye = keypoints_with_scores[0][0][2] # [x-coordinate, y-coordinate, Confidence Level]
left_elbow = keypoints_with_scores[0][0][7]

In [56]:
# To get the real coordinate of left_elbow
np.array(left_elbow[:2]*[480,640]).astype(int)

array([375, 582])